In [ ]:
import numpy as np
import data_helpers
from w2v import train_word2vec

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input, MaxPooling1D, Convolution1D, Embedding
from keras.layers.merge import Concatenate
from keras.datasets import imdb
from keras.preprocessing import sequence
np.random.seed(0)

In [ ]:
# ---------------------- Parameters section -------------------
#
# Model type. See Kim Yoon's Convolutional Neural Networks for Sentence Classification, Section 3
model_type = "CNN-rand"  # CNN-rand|CNN-non-static|CNN-static

# Model Hyperparameters
embedding_dim = 50
filter_sizes = (3, 8)
num_filters = 10
dropout_prob = (0.5, 0.8)
hidden_dims = 50

# Training parameters
batch_size = 64
num_epochs = 10

# Prepossessing parameters
sequence_length = 400
max_words = 5000

# Word2Vec parameters (see train_word2vec)
min_word_count = 1
context = 10
#
# ---------------------- Parameters end -----------------------

In [8]:
# Data Preparation
print("Load data...")
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_words, start_char=None)

x_train = sequence.pad_sequences(x_train, maxlen=sequence_length, padding="post", truncating="post")
x_test = sequence.pad_sequences(x_test, maxlen=sequence_length, padding="post", truncating="post")
print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)

vocabulary = imdb.get_word_index()
vocabulary_inv = dict((v, k) for k, v in vocabulary.items())
vocabulary_inv[0] = "<PAD/>"


# Prepare embedding layer weights and convert inputs for static model
print("Model type is", model_type)
if model_type == "CNN-non-static" or model_type == "CNN-static":
    embedding_weights = train_word2vec(np.vstack((x_train, x_test)), vocabulary_inv, num_features=embedding_dim,
                                       min_word_count=min_word_count, context=context)
    if model_type == "CNN-static":
        x_train = embedding_weights[0][x_train]
        x_test = embedding_weights[0][x_test]
elif model_type == "CNN-rand":
    embedding_weights = None
else:
    raise ValueError("Unknown model type")


# Build model
input_shape = (sequence_length, embedding_dim) if model_type == "CNN-static" else (sequence_length,)
model_input = Input(shape=input_shape)

# Static model do not have embedding layer
if model_type == "CNN-static":
    z = Dropout(dropout_prob[0])(model_input)
else:
    z = Embedding(len(vocabulary_inv), embedding_dim, input_length=sequence_length, name="embedding")(model_input)
    z = Dropout(dropout_prob[0])(z)

# Convolutional block
conv_blocks = []
for sz in filter_sizes:
    conv = Convolution1D(filters=num_filters,
                         kernel_size=sz,
                         padding="valid",
                         activation="relu",
                         strides=1)(z)
    conv = MaxPooling1D(pool_size=2)(conv)
    conv = Flatten()(conv)
    conv_blocks.append(conv)
z = Concatenate()(conv_blocks) if len(conv_blocks) > 1 else conv_blocks[0]

z = Dropout(dropout_prob[1])(z)
z = Dense(hidden_dims, activation="relu")(z)
model_output = Dense(1, activation="sigmoid")(z)

model = Model(model_input, model_output)
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Initialize weights with word2vec
if model_type == "CNN-non-static":
    embedding_layer = model.get_layer("embedding")
    embedding_layer.set_weights(embedding_weights)

# Train the model
model.fit(x_train, y_train, batch_size=batch_size, epochs=num_epochs,
          validation_data=(x_test, y_test), verbose=2)

Load data...
x_train shape: (25000, 400)
x_test shape: (25000, 400)
Model type is CNN-rand
Train on 25000 samples, validate on 25000 samples
Epoch 1/10
47s - loss: 0.6047 - acc: 0.6160 - val_loss: 0.3351 - val_acc: 0.8609
Epoch 2/10
42s - loss: 0.3071 - acc: 0.8724 - val_loss: 0.2760 - val_acc: 0.8838
Epoch 3/10
39s - loss: 0.2571 - acc: 0.8978 - val_loss: 0.2812 - val_acc: 0.8832
Epoch 4/10
39s - loss: 0.2352 - acc: 0.9084 - val_loss: 0.2802 - val_acc: 0.8839
Epoch 5/10
41s - loss: 0.2213 - acc: 0.9120 - val_loss: 0.2751 - val_acc: 0.8872
Epoch 6/10
38s - loss: 0.2132 - acc: 0.9156 - val_loss: 0.2767 - val_acc: 0.8846
Epoch 7/10
39s - loss: 0.1939 - acc: 0.9246 - val_loss: 0.2893 - val_acc: 0.8826
Epoch 8/10
39s - loss: 0.1895 - acc: 0.9237 - val_loss: 0.2966 - val_acc: 0.8806
Epoch 9/10
38s - loss: 0.1855 - acc: 0.9280 - val_loss: 0.3016 - val_acc: 0.8814
Epoch 10/10
39s - loss: 0.1781 - acc: 0.9280 - val_loss: 0.2991 - val_acc: 0.8804
